In [1]:
from mmdet.apis import init_detector, inference_detector
from mmdet.utils import register_all_modules
from mmdet.registry import VISUALIZERS
import mmcv
from pycocotools import mask
import json
import numpy as np
import os
import glob
from skimage import measure
from shapely.geometry import Polygon,MultiPolygon

In [2]:
# download the checkpoint demo




# !mim download mmdet --config rtmdet_tiny_8xb32-300e_coco --dest ./checkpoints
config_file = '/home/sbhandari/seg/mmdetection/imp_checkpoints/new_sub/original_convnext_v2_mask_rcnn.py'
checkpoint_file = '/home/sbhandari/seg/mmdetection/imp_checkpoints/new_sub/best_coco_segm_mAP_50_epoch_33.pth'

In [3]:
#Register all modules in mmdet into the registries
register_all_modules()
# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')  # or device='cpu'

Loads checkpoint by local backend from path: /home/sbhandari/seg/mmdetection/imp_checkpoints/new_sub/best_coco_segm_mAP_50_epoch_33.pth


In [4]:
with open("/home/sbhandari/seg/mmdetection/empt_ann.json") as file:
    gt = json.load(file)

empty_submit = dict()
empty_submit["images"] = []
empty_submit["categories"] = gt["categories"]
empty_submit["annotations"] = []

In [5]:
count_anns = 1
images = glob.glob("/home/sbhandari/testcasses/test_cases_stenosis/images/" + "*.png")
# print(images)
for image in images:
    # print(image)
    img = mmcv.imread( image, channel_order='rgb')
    image_id = int(os.path.splitext(os.path.basename(image))[0])
    empty_submit["images"].append({'id': image_id, 'width': 512, 'height': 512, 'license':0,'date_captured':str(0), 'file_name': str(image_id)+'.png'})
    result = inference_detector(model, img)
    print(f" Inferred {image_id} ")
    masks = result.pred_instances.masks.cpu().numpy()
    scores = result.pred_instances.scores.cpu().numpy()
    labels = result.pred_instances.labels.cpu().numpy()
    boxes = result.pred_instances.bboxes.cpu().numpy() 
    indexes = np.flip(np.argsort(scores))
    print(indexes)
    count = 0   
    while count<3 and count < len(indexes):#or indexes.size != 0 :
        
        index = indexes[count]
        score = scores[index]
        count += 1
        if score>0.8:
            mask = masks[index]
            label= labels[index]
            box = boxes[index]
            contours = measure.find_contours(mask)
            
            for contour in contours:             
                    for i in range(len(contour)):
                        row, col = contour[i]
                        contour[i] = (col - 1, row - 1)

                    # Simplify polygon
                    poly = Polygon(contour)
                    poly = poly.simplify(1.0, preserve_topology=False)
            
                    if(poly.is_empty):
                        continue
                    segmentation = np.array(poly.exterior.coords).ravel().tolist()
                    new_ann = dict()
                    new_ann["id"] = count_anns
                    new_ann["image_id"] = image_id
                    new_ann["category_id"] = (label+26).tolist()
                    new_ann["segmentation"] = [segmentation]
                    area = poly.area
                    new_ann["area"] = area
                    x, y = contour.min(axis=0)
                    w, h = contour.max(axis=0) - contour.min(axis=0)
                    new_ann["bbox"]  = [int(x), int(y), int(w), int(h)]
                    new_ann["iscrowd"] = 0
                    new_ann["attributes"] = {
                        "occluded": False
                    }
                    
                    # if int(area)>90:
                    count_anns += 1
                    empty_submit["annotations"].append(new_ann.copy())

    #

 Inferred 61 
[0 1 2 3 4 5 6]
 Inferred 139 
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
 Inferred 55 
[0 1 2 3 4 5 6 7 8 9]
 Inferred 135 
[ 0  1  2  3  4  5  6  7  8  9 10 11]
 Inferred 53 
[0 1 2 3 4 5]
 Inferred 62 
[ 0  1  2  3  4  5  6  7  8  9 10 11 12]
 Inferred 97 
[ 0  1  2  3  4  5  6  7  8  9 10]
 Inferred 51 
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 Inferred 103 
[0 1 2 3 4 5 6]
 Inferred 2 
[ 0  1  2  3  4  5  6  7  8  9 10 11]
 Inferred 214 
[ 0  1  2  3  4  5  6  7  8  9 10]
 Inferred 149 
[0 1 2 3 4 5 6 7 8]
 Inferred 90 
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
 Inferred 281 
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 Inferred 94 
[ 0  1  2  3  4  5  6  7  8  9 10 11]
 Inferred 29 
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
 Inferred 300 
[ 0  1  2  3  4  5  6  7  8  9 10 11 12]
 Inferred 292 
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
 Inferred 154 
[0 1 2 3 4 5 6]
 Inferred 130 
[ 0  1  2  3  4  5  6  7  8  9 10 11 12]
 In

In [6]:
with open("input_coco.json", "w") as file:
    json.dump(empty_submit, file)